# Data Scraping for Binance Announcements
Beta version. Modified on 07-09-2019

Import all the needed packages:

In [1]:
import bs4 as bs
import urllib.request
import tweepy, os, time

Twitter app

In [2]:
consumer_key = os.environ.get('TW_CONSUMER_KEY')
consumer_secret = os.environ.get('TW_CONSUMER_SECRET')
access_token = os.environ.get('TW_ACCESS_TOKEN')
access_token_secret = os.environ.get('TW_ACCESS_TOKEN_SECRET')
# authentication of consumer key and secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# authentication of access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Create two empty lists for storing news urls. Message should be sent when there is a new item in the news_urls that wasn't in the old_urls

In [3]:
old_urls, news_urls = [], []

Create a bag of key words for getting matches. Don't use plurals, otherwise will get duplicates

In [4]:
key_words = ['List', 'list', 'Token Sale', 'Open Trading', 'open trading']

Create the function to extract the information from the webpage and get the matchings

In [5]:
def extract_binance(main_webpage, key_words):
    final_item, final_list = [], []
    sauce = urllib.request.urlopen(main_webpage).read()
    soup = bs.BeautifulSoup(sauce, 'lxml')
    list = soup.find_all('li', class_ = 'article-list-item')
    for article in list:
        article_text = article.get_text().replace('\n', '')
        for item in key_words:
            if item in article_text:
                final_item.append(article_text)
                final_item.append('https://www.binance.com' + article.find('a').get('href'))
                final_list.append(final_item)
                final_item = [] # Reset once is in the final_list to not get duplicates
    return final_list

Get the first pass

In [6]:
%%time
main_webpage = 'https://www.binance.com/en/support/categories/115000056351'
old_urls = extract_binance(main_webpage, key_words)

CPU times: user 30.9 ms, sys: 349 µs, total: 31.2 ms
Wall time: 77.1 ms


Loop pass and get the new announcements

In [ ]:
# Loop pass - Watchdog mode
while True:
    new_urls = extract_binance(main_webpage, key_words)
    for item in new_urls:
        if item not in old_urls:
            msg = item[0] + '\n' + item[1]
            api.update_status(msg)
    print('Done for now. Time to go to sleep mate!')
    time.sleep(900) # Check every 15 min

Done for now. Time to go to sleep mate!
